<a href="https://colab.research.google.com/github/svmamidi/DS-Project/blob/main/HCT_survival_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

equity_post_hct_survival_predictions_path = kagglehub.competition_download('equity-post-HCT-survival-predictions')

print('Data source import complete.')


100%|██████████| 1.27M/1.27M [00:00<00:00, 2.38MB/s]

Extracting files...
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install lifelines


In [ ]:


import pandas as pd
from lifelines import KaplanMeierFitter


In [ ]:
train = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
test = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/equity-post-HCT-survival-predictions/train.csv'

In [ ]:
train['efs'] = train['efs'].astype('category')
train.head()

NameError: name 'train' is not defined

In [ ]:
def kaplan(train, time_col = 'efs_time', event_col='efs'):

    kmf = KaplanMeierFitter()
    kmf.fit(train[time_col], event_observed=train[event_col])
    return kmf.survival_function_at_times(train[time_col]).values.flatten()


train['target'] = kaplan(train=train)
train['target']

In [ ]:
train_data_features = train.drop(columns=['efs', 'efs_time'])

In [ ]:
train.dtypes

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
data = pd.concat([train_data_features, test], axis=0).reset_index(drop=True)


In [ ]:
data.head

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
data.dtypes

In [ ]:
data.describe().T

In [ ]:
data.isnull().sum()

In [ ]:
duplicates = data.duplicated()
print(duplicates)

In [ ]:
num_duplicates = data.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")
# data.drop('ID', axis=1, inplace=True)

In [ ]:
numerical_cols = data.select_dtypes(include=['int', 'float']).columns

numerical_cols

In [ ]:
data.dtypes

In [ ]:
# Identify categorical columns
categorical_cols = data.select_dtypes(include=['object']).columns

# Remove whitespaces and standardize text
for col in categorical_cols:
    data[col] = data[col].str.strip().str.lower()


# Verify changes
for col in categorical_cols:
    unique_values = data[col].unique()
    print(f"{col}: {unique_values}\n")

In [ ]:
for col in categorical_cols:
    data[col] = data[col].astype('category')

data.info()

In [ ]:
categorical_cols = data.select_dtypes(include=['category']).columns
categorical_cols

In [ ]:
numerical_cat_columns = ['hla_match_c_high', 'hla_high_res_8', 'hla_low_res_6',
                     'hla_high_res_6', 'hla_high_res_10', 'hla_match_dqb1_high',
                     'hla_nmdp_6', 'hla_match_c_low', 'hla_match_drb1_low',
                     'hla_match_dqb1_low', 'hla_match_a_high',
                      'hla_match_b_low',
                     'hla_match_a_low', 'hla_match_b_high', 'comorbidity_score',
                     'karnofsky_score', 'hla_low_res_8', 'hla_match_drb1_high',
                     'hla_low_res_10','dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'tbi_status',
               'arrhythmia', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe',
               'prim_disease_hct', 'cmv_status', 'tce_imm_match', 'rituximab',
               'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity',
               'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hepatic_severe',
               'prior_tumor', 'peptic_ulcer', 'gvhd_proph', 'rheum_issue', 'sex_match',
               'race_group', 'hepatic_mild', 'tce_div_match', 'donor_related',
               'melphalan_dose', 'cardiac', 'pulm_moderate']

#  Convert numerical columns to categorical
for col in numerical_cat_columns:
    data[col] = data[col].astype("category")

In [ ]:
data.dtypes

In [ ]:

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [ ]:
#Replace blanks with NaN
data[numerical_cat_columns] = data[numerical_cat_columns].replace(r'^\s*$', np.nan, regex=True)

#  Label Encoding for categorical variables
label_encoders = {}
for col in numerical_cat_columns:
    le = LabelEncoder()
    data[col] = data[col].astype(str)  # Convert to string to ensure encoding works
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le  # Store encoders for reverse transformation later

#  KNN Imputation
imputer = KNNImputer(n_neighbors=5, weights="uniform")  # Adjust n_neighbors as needed
data[numerical_cat_columns] = imputer.fit_transform(data[numerical_cat_columns])

#  Convert columns back to integers
for col in numerical_cat_columns:
    data[col] = data[col].round(0).astype(int)  # Round off and convert to integers


In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
for col in numerical_cat_columns:
    data[col] = data[col].astype("category")

In [ ]:
data.dtypes

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
import matplotlib.pyplot as plt


# Assuming your data is in a DataFrame called df
# Replace 'donor_age' with your actual column name
plt.figure(figsize=(8, 6))

# Plot the histogram
plt.hist(data['donor_age'].dropna(), bins=20, color='skyblue', edgecolor='black')

# Add labels and title
plt.xlabel('Donor Age')
plt.ylabel('Frequency')
plt.title('Distribution of Donor Age')

# Show the plot
plt.show()

In [ ]:
# Calculate the mean of 'donor_age' excluding NaN
mean_donor_age = data['donor_age'].mean()

# Impute missing values with the mean
data['donor_age'].fillna(mean_donor_age, inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
# Convert to datetime
data['year_hct'] = pd.to_datetime(data['year_hct'], format='%Y', errors='coerce')

In [ ]:
data.dtypes

In [ ]:
data['year_hct'] = pd.to_datetime(data['year_hct'])

# Extract only the year
data['year_hct'] = data['year_hct'].dt.year

In [ ]:
ids_to_remove = [28800,28801,28802]
test_data = data[data['ID'].isin(ids_to_remove)]
x_train = data[~data['ID'].isin(ids_to_remove)]

In [ ]:
test_data=test_data.drop(columns=['target'])

In [ ]:
pd.set_option('display.max_columns', None)
x_train.head()

In [ ]:
features = [
    'year_hct', 'age_at_hct', 'donor_age', 'conditioning_intensity', 'prim_disease_hct',
    'gvhd_proph', 'dri_score', 'race_group', 'comorbidity_score', 'sex_match', 'cmv_status',
    'cyto_score', 'karnofsky_score', 'tce_imm_match', 'tbi_status', 'cyto_score_detail',
    'tce_div_match', 'tce_match', 'mrd_hct', 'in_vivo_tcd', 'melphalan_dose',
    'pulm_moderate', 'ethnicity', 'psych_disturb', 'donor_related', 'diabetes', 'hepatic_severe',
    'hla_high_res_10', 'prior_tumor', 'hla_nmdp_6', 'prod_type', 'graft_type', 'hla_low_res_10',
    'cardiac', 'pulm_severe', 'hepatic_mild', 'obesity', 'arrhythmia','hla_high_res_8','hla_low_res_6',
    'hla_match_c_high','hla_high_res_6','hla_match_dqb1_high','hla_match_c_low',
    'hla_high_res_10',
                    'hla_nmdp_6', ,
                     'hla_match_dqb1_low', 'hla_match_a_high',
                      'hla_match_b_low',
                     'hla_match_a_low', 'hla_match_b_high'



]

In [ ]:
# features=['dri_score', 'psych_disturb', 'cyto_score', 'diabetes',
#        'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia',
#        'hla_low_res_6', 'graft_type',  'prim_disease_hct', 'hla_high_res_6', 'cmv_status',
#        'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6',
#        'hla_match_c_low',  'hla_match_drb1_low',
#        'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail',
#        'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct',
#        'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe',
#        'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer',
#        'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue',
#        'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score',
#        'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related',
#        'melphalan_dose', 'hla_low_res_8', 'cardiac', 'hla_match_drb1_high',
#        'pulm_moderate', 'hla_low_res_10','vent_hist','renal_issue','pulm_severe','rituximab']

In [ ]:
X = x_train[features]
X = x_train.reset_index(drop=True)
y = x_train['target']
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

# Correct splitting
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Check shapes
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")


In [ ]:
from catboost import CatBoostRegressor

In [ ]:
# model1 = CatBoostRegressor(cat_features=numerical_cat_columns,loss_function='MultiRMSE', eval_metric='MultiRMSE', iterations=100, learning_rate=0.1, depth=6)
# model1.fit(X_train,y_train,eval_set=(X_val, y_val), verbose=True)

In [ ]:
from sklearn.ensemble import StackingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import lightgbm as lgb


# Define the base models
base_models = [
    ('rf', RandomForestRegressor(n_estimators=200, random_state=42)),
    ('gb', GradientBoostingRegressor(n_estimators=100, random_state=42)),
    ('dt', DecisionTreeRegressor(random_state=42)),
    ('lgb', lgb.LGBMRegressor(n_estimators=300, random_state=42))

]

# Define the meta-model
meta_model = Ridge()

model2 = StackingRegressor(estimators=base_models, final_estimator=meta_model)
model2.fit(X_train, y_train)
ensemble_pred = model2.predict(X_val)

In [ ]:
# Evaluating model performance: C-index
from lifelines.utils import concordance_index
c_index = concordance_index(y_val, ensemble_pred, event_observed=y_val)
print(f"C-index on Validation Set: {c_index}")

In [ ]:
test_data=test_data.drop(columns=['target','ID'], errors='ignore')
test_data=test_data[features]
prediction = model2.predict(test_data)

In [ ]:
final_predictions = pd.DataFrame(
    {
        "ID": test_data.index,
        "prediction": prediction,
    }
)
final_predictions.to_csv("submission.csv", index=False)
print("Final predictions saved to 'submission.csv'.")

In [ ]:
final_predictions